## Importing Data and Libraries

In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score, 
    recall_score, 
    precision_score, 
    f1_score, 
    cohen_kappa_score
)
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    cohen_kappa_score,
)



In [23]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

X_train = Train.drop('STATUS', axis=1)
y_train = Train['STATUS']

X_test = Test.drop('STATUS', axis=1)
y_test = Test['STATUS']

In [24]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.array
y_test = y_test.array

## Building the model

### Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer

def train_logistic_regression(X, y):
    # Define a simplified hyperparameter grid
    param_grid = {
        "penalty": ["l2"],
        "solver": ["lbfgs"],  # Default solver for `l2`
        "C": [1, 10],        # Fewer options for regularization strength
    }

    # Define a single scoring metric
    scoring = make_scorer(accuracy_score)

    # Initialize Logistic Regression
    model = LogisticRegression(max_iter=500)  # Adjust iterations if necessary

    # Perform Grid Search with fewer folds
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=scoring,
        n_jobs=-1,
        cv=3,                # Reduced cross-validation folds
        verbose=1
    )

    # Fit the grid search
    grid_search.fit(X, y)

    # Output best parameters and metrics
    print("Best Parameters:", grid_search.best_params_)
    print("Best Accuracy:", grid_search.best_score_)

    return grid_search.best_estimator_


### RFC

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer

def train_random_forest(X, y):
    # Simplified hyperparameter grid
    param_grid = {
        "min_samples_split": [2, 10],
        "min_samples_leaf": [1, 5],
        "n_estimators": [100, 200],
        "max_depth": [None, 10]
    }

    # Define the scoring metric
    scoring = make_scorer(accuracy_score)

    # Initialize the model
    model = RandomForestClassifier(random_state=1)

    # GridSearchCV with fewer folds
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=scoring,
        n_jobs=-1,
        cv=3,  # Reduced cross-validation splits
        verbose=1
    )

    # Fit the grid search
    grid_search.fit(X, y)

    # Output the best parameters and the best score
    print("Best Parameters:", grid_search.best_params_)
    print("Best Accuracy:", grid_search.best_score_)

    return grid_search.best_estimator_


SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer

def train_svm(X, y):
    # Define a simplified hyperparameter grid
    param_grid = {
        "C": [0.1, 1, 10],
        "gamma": ["scale", "auto"],
        "kernel": ["rbf", "linear", "poly"],
        "degree": [3, 4]  # Relevant only for `poly` kernel
    }

    # Define scoring metric
    scoring = make_scorer(accuracy_score)

    # Initialize the model
    model = SVC(class_weight="balanced", random_state=1)

    # Perform Grid Search with cross-validation
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=scoring,
        n_jobs=-1,
        cv=3,  # Reduced cross-validation folds
        verbose=1
    )

    # Fit the grid search
    grid_search.fit(X, y)

    # Output best parameters and metrics
    print("Best Parameters:", grid_search.best_params_)
    print("Best Accuracy:", grid_search.best_score_)

    return grid_search.best_estimator_


In [28]:
train_logistic_regression(X_train, y_train)

train_random_forest(X_train, y_train)

train_svm(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Accuracy: 0.874292266896008
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Accuracy: 0.9585035476241668
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'C': 10, 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf'}
Best Accuracy: 0.9512649609402996
